### Monitoring wetland dynamics and its impact on malaria in the Zambesi delta, Mozambique

#### Context
What are wetlands?, why important?  
Why monitor?  
Relationship to malaria? Why it matters?  

### Method Description
1. Tasselled-Cap Wetnees (TCW) from surface reflectance
2. Fractional Cover
3. Masking using WOfS and TCW
4. Stacking and classifying the dominant land type
5. Comparison to malaria incidence rate 

#### 1. Tasseled-Cap Wetness(TCW) computaion from surface reflectance
TCW is one of the Tasseled-Cap Transform (TCT) indices, which uses a linear combination of Landsat bands to detect moisture content in soil and vegetation. The equations vary by Landsat sensor:

Landsat 5 & 7 (TM) Coefficients  
TCW=0.1509B1+0.1973B2+0.3279B3+0.3406B4−0.7112B5−0.4572B7  
Landsat 7 (ETM+) Coefficients  
TCW=0.2626B1+0.2141B2+0.0926B3+0.0656B4−0.7629B5−0.5388B7  
Landsat 8 & 9 (OLI) Coefficients  
TCW=0.1511𝐵2+0.1973𝐵3+0.3283𝐵4+0.3407𝐵5−0.7117𝐵6−0.4559𝐵7  
TCW=0.1511B2+0.1973B3+0.3283B4+0.3407B5−0.7117B6−0.4559B7  

B1, B2, etc., are Landsat bands (Surface Reflectance)

TCW is sensitive to water content in soil and vegetation

Higher TCW means more moisture, lower TCW means drier surfaces

In [1]:
import datacube
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import rgb

In [2]:
#connect to datacube
dc = datacube.Datacube(app="Landsat_Surface_Reflectance")

In [3]:
# List Landsat products available in DE Africa
dc_products = dc.list_products()
display_columns = ['name', 'description']
dc_products[dc_products.name.str.contains(
    'sr').fillna(
        False)][display_columns].set_index('name')

,description
name,
dem_srtm,1 second elevation model
dem_srtm_deriv,1 second elevation model derivatives
ls5_sr,USGS Landsat 5 Collection 2 Level-2 Surface Re...
ls7_sr,USGS Landsat 7 Collection 2 Level-2 Surface Re...
ls8_sr,USGS Landsat 8 Collection 2 Level-2 Surface Re...
ls9_sr,USGS Landsat 9 Collection 2 Level-2 Surface Re...


In [4]:
dc_measurements = dc.list_measurements()
# Display available measurements for Landsat 5, 7, 8, and 9
landsat_collections = ["ls8_sr", "ls7_sr", "ls5_sr"] 
for collection in landsat_collections:
    collection in dc_measurements.index
    print(f"Measurements for {collection}:")
    display(dc_measurements.loc[collection])

Measurements for ls8_sr:


,name,dtype,units,nodata,aliases,flags_definition,add_offset,scale_factor
measurement,,,,,,,,
SR_B1,SR_B1,uint16,1,0.0,"[band_1, coastal_aerosol]",NaN,NaN,NaN
SR_B2,SR_B2,uint16,1,0.0,"[band_2, blue]",NaN,NaN,NaN
SR_B3,SR_B3,uint16,1,0.0,"[band_3, green]",NaN,NaN,NaN
SR_B4,SR_B4,uint16,1,0.0,"[band_4, red]",NaN,NaN,NaN
SR_B5,SR_B5,uint16,1,0.0,"[band_5, nir]",NaN,NaN,NaN
SR_B6,SR_B6,uint16,1,0.0,"[band_6, swir_1]",NaN,NaN,NaN
SR_B7,SR_B7,uint16,1,0.0,"[band_7, swir_2]",NaN,NaN,NaN
QA_PIXEL,QA_PIXEL,uint16,bit_index,1.0,"[pq, pixel_quality]","{'snow': {'bits': 5, 'values': {'0': 'not_high...",NaN,NaN
QA_RADSAT,QA_RADSAT,uint16,bit_index,0.0,"[radsat, radiometric_saturation]","{'nir_saturation': {'bits': 4, 'values': {'0':...",NaN,NaN


Measurements for ls7_sr:


,name,dtype,units,nodata,aliases,flags_definition,add_offset,scale_factor
measurement,,,,,,,,
SR_B1,SR_B1,uint16,1,0.0,"[band_1, blue]",NaN,NaN,NaN
SR_B2,SR_B2,uint16,1,0.0,"[band_2, green]",NaN,NaN,NaN
SR_B3,SR_B3,uint16,1,0.0,"[band_3, red]",NaN,NaN,NaN
SR_B4,SR_B4,uint16,1,0.0,"[band_4, nir]",NaN,NaN,NaN
SR_B5,SR_B5,uint16,1,0.0,"[band_5, swir_1]",NaN,NaN,NaN
SR_B7,SR_B7,uint16,1,0.0,"[band_7, swir_2]",NaN,NaN,NaN
QA_PIXEL,QA_PIXEL,uint16,bit_index,1.0,"[pq, pixel_quality]","{'snow': {'bits': 5, 'values': {'0': 'not_high...",NaN,NaN
QA_RADSAT,QA_RADSAT,uint16,bit_index,0.0,"[radsat, radiometric_saturation]","{'dropped_pixel': {'bits': 9, 'values': {'0': ...",NaN,NaN
SR_ATMOS_OPACITY,SR_ATMOS_OPACITY,int16,1,-9999.0,[atmos_opacity],NaN,NaN,NaN


Measurements for ls5_sr:


,name,dtype,units,nodata,aliases,flags_definition,add_offset,scale_factor
measurement,,,,,,,,
SR_B1,SR_B1,uint16,1,0.0,"[band_1, blue]",NaN,NaN,NaN
SR_B2,SR_B2,uint16,1,0.0,"[band_2, green]",NaN,NaN,NaN
SR_B3,SR_B3,uint16,1,0.0,"[band_3, red]",NaN,NaN,NaN
SR_B4,SR_B4,uint16,1,0.0,"[band_4, nir]",NaN,NaN,NaN
SR_B5,SR_B5,uint16,1,0.0,"[band_5, swir_1]",NaN,NaN,NaN
SR_B7,SR_B7,uint16,1,0.0,"[band_7, swir_2]",NaN,NaN,NaN
QA_PIXEL,QA_PIXEL,uint16,bit_index,1.0,"[pq, pixel_quality]","{'snow': {'bits': 5, 'values': {'0': 'not_high...",NaN,NaN
QA_RADSAT,QA_RADSAT,uint16,bit_index,0.0,"[radsat, radiometric_saturation]","{'dropped_pixel': {'bits': 9, 'values': {'0': ...",NaN,NaN
SR_ATMOS_OPACITY,SR_ATMOS_OPACITY,int16,1,-9999.0,[atmos_opacity],NaN,NaN,NaN


In [5]:
# bounding box for Mopeia in Zambezi delta Mozambique
lat, lon =  -17.9782, 35.7136
buffer = 0.245 

#add lat,lon,buffer together to get bounding box
x = (lon-buffer, lon+buffer)
y =  (lat+buffer, lat-buffer)

print(x,y)

(35.4686, 35.9586) (-17.7332, -18.223200000000002)


In [ ]:
# Load ARD (Landsat 5-9) data for the specified time range (2000-2022)
#maybe try 1 year
# Create a reusable query
query = {
    'x': x,
    'y': y,
    'time': ("2020-01","2022-01"),
}

#get crs of location
verbose_progress = True
if verbose_progress:
        print("Loading Landsat data")
    
ds_ls = load_ard(
    dc=dc,
    products=["ls8_sr", "ls7_sr", "ls5_sr"],
    output_crs="epsg:6933",
    #min_gooddata=0.85,
    mask_filters=(["opening", 3], ["dilation", 3]),
    measurements=["red", "green", "blue", "nir", "swir_1", "swir_2"],
    #dask_chunks=None,
    group_by="solar_day",
    resolution=(-30, 30),
    #verbose=False,
    **query,
)
print(ds_ls)

Loading Landsat data
Using pixel quality parameters for USGS Collection 2
Finding datasets
    ls8_sr
    ls7_sr
    ls5_sr
Applying morphological filters to pq mask (['opening', 3], ['dilation', 3])
Applying pixel quality/cloud mask
Re-scaling Landsat C2 data
Loading 171 time steps


In [ ]:
# plotting single band images
#selecting and plotting a single timestep

first_timestep = ds.swir_1.isel(time=0) #.isel() = index selection 
print(first_timestep)

In [ ]:
def tasseled_cap_wetness(ds, mission):
    if mission == 'ls5' :  # Landsat 5 coefficients
        coefficients = {
            'blue': 0.3037,'green': 0.2793,'red': 0.4743,'nir': 0.5585,'swir1': 0.5082,'swir2': 0.1863
        }
    elif mission == 'ls8' or mission == 'ls9':  # Landsat 8 coefficients
        coefficients = {
            'blue': 0.2630,'green': 0.1456, 'red': 0.3379,'nir': 0.4556,'swir1': 0.6140,'swir2': 0.1126
        }
    elif mission == 'ls8' or mission == 'ls9':  # Landsat 8 & 9 coefficients
        coefficients = {
            'blue': 0.2630,'green': 0.1456,'red': 0.3379,'nir': 0.4556,'swir1': 0.6140,'swir2': 0.1126
        }
    else:
        raise ValueError("Unsupported Landsat mission")

    # Calculate Tasseled Cap Wetness (TCW) for the given mission
    wetness = (ds['blue'] * coefficients['blue'] + ds['green'] * coefficients['green'] + ds['red'] * coefficients['red'] +
               ds['nir'] * coefficients['nir'] + ds['swir1'] * coefficients['swir1'] + ds['swir2'] * coefficients['swir2'])
    return wetness

# Apply the Tasseled Cap Wetness calculation to each dataset
wetness_ds = ds.map(lambda x: tasseled_cap_wetness(x, x.attrs['product']))
print(wetness_ds)

#### 2. Fractional Cover (FC)

#### 3. Masking using WOfS and TCW

#### 4. Stacking and classifying land type

#### 5. compare statistics: water inundation vs malaria incidence rate